In [10]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
import os 


def Normlize(Z):
    Zmax, Zmin = Z.max(axis=1), Z.min(axis=1)
    Zmean = Z.mean(axis=1)
    #按列排序
    Zmax, Zmin = Zmax.reshape(-1, 1), Zmin.reshape(-1, 1)
    Zmean = Zmean.reshape(-1, 1)
    Z = (Z - Zmean) / (Zmax - Zmin)
    return Z


def Data_Reading(Normalization=True):
    train_x = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\train\\trainset2.npy')
    train_y = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\train\\trainlabel2.npy')
    test_x = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\test\\testset2.npy')
    test_y = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\test\\testlabel2.npy')

    # Normalization
    train_x_Normed = Normlize(train_x)
    test_x_Normed = Normlize(test_x)

    # xlsx to tensor
    if Normalization:
        train_x = torch.from_numpy(train_x_Normed).type(torch.cuda.FloatTensor)
        test_x = torch.from_numpy(test_x_Normed).type(torch.cuda.FloatTensor)
        train_y = torch.from_numpy(train_y).type(torch.int64)
        test_y = torch.from_numpy(test_y).type(torch.int64)

    else:
        train_x = torch.from_numpy(train_x).type(torch.FloatTensor)
        test_x = torch.from_numpy(test_x).type(torch.FloatTensor)
        train_y = torch.from_numpy(train_y).type(torch.int64)
        test_y = torch.from_numpy(test_y).type(torch.int64)
    # reshape
    train_x = train_x.view(410, 1, 10, 120)
    test_x = test_x.view(174, 1, 10, 120)
    # train_x = train_x.view(408, 1, 10, 120)
    # test_x = test_x.view(202, 1, 10, 120)

    return train_x, test_x, train_y, test_y


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,3,(1,3),stride=(1,1))#(10 - 1)/1 + 1 = 10   (120 - 3)/1 + 1 = 118 118/2 = 59
        self.conv2 = nn.Conv2d(3,6,(2,3),stride=(2,1))#(10 - 2)/2 + 1 = 5   (59 - 3)/1 + 1 = 57 
        self.conv3 = nn.Conv2d(6,10,(5,3),stride=(1,1))# (5 - 5)/1 + 1 = 1  (57 - 3)/1 + 1 = 55
        self.fc1 = nn.Linear(10*1*55,4)

       
        # self.conv1 = nn.Conv2d(1,6,(10,4),stride=(1,2))#59
        # self.conv2 = nn.Conv2d(6,10,(1,3),stride=(1,1))#57      
        # self.fc1 = nn.Linear(10*1*57,4)

        # self.fc2 = nn.Linear(342,4)
        # self.fc3 = nn.Linear(85,4)
        # self.fc4 = nn.Linear(64,4)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.xavier_uniform_(m.weight)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.01)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, (1,2))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        # x = F.logsigmoid(self.fc1(x))
        # x = F.logsigmoid(self.fc2(x))
        # x = F.relu(self.fc3(x))
        x = self.fc1(x)
        # return x
        return x#F.softmax(x,dim=0)


if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")#cuda:0

    print(device)

    cnn = Net()
    print(cnn)
    cnn.to(device)

    #sgd -> stochastic gradient descent
    optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.6)#
    loss_func = nn.CrossEntropyLoss()

    train_x, test_x, train_y, test_y = Data_Reading(Normalization=1)
    train_y = train_y.squeeze()
    test_y = test_y.squeeze()
    train_x = train_x.to(device)
    test_x = test_x.to(device)
    train_y = train_y.to(device)
    test_y = test_y.to(device)

    

    #train
    sum = 0

    batch_size = 17
    step = 10
    tr_x = Variable(train_x)
    tr_y = Variable(train_y)
    for epoch in range(200):
        running_loss = 0.0
        for i in range(0,(int)(len(train_x)/batch_size)):
            t_x = Variable(train_x[i*batch_size:i*batch_size+batch_size])
            t_y = Variable(train_y[i*batch_size:i*batch_size+batch_size])
            out = cnn(t_x)
            #forward
            loss = loss_func(out, t_y)
            #梯度初始化为零
            optimizer.zero_grad()
             
            #backward
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # print('loss = {}, i = {}'.format(loss.item(), i))

            # predicted1 = torch.max(out1.data, 1)[1]
   
        running_loss = running_loss / 24

        out = cnn(tr_x)
        predicted_train = torch.max(out.data, 1)[1]
        total_train = tr_y.size(0)
        for j in range(tr_y.size(0)):
            if predicted_train[j] == tr_y[j]:
                sum += 1
        
        if (epoch + 1) % 2 == 0:
            print('total_train:{}, accuracy:{}, sum:{}'.format(total_train, sum / total_train, sum))
        sum = 0

        if (sum / total_train > 0.85) :
            optimizer = optim.SGD(cnn.parameters(), lr=0.0001, momentum=0.4)
        elif (sum / total_train > 0.95) :
            optimizer = optim.SGD(cnn.parameters(), lr=0.00001, momentum=0.2)

        if (epoch + 1) % 2 == 0:
            print('Epoch[{}], loss: {:.8f}'.format(epoch + 1, running_loss))
    
    
    #test
        te_x = Variable(test_x)
        te_y = Variable(test_y)
        out1 = cnn(te_x)
        predicted_test = torch.max(out1.data, 1)[1]#.data.squeeze()
        total = te_y.size(0)

        for j in range(te_y.size(0)):
            if predicted_test[j] == te_y[j]:
                sum += 1

        print('total:{}, accuracy:{}, sum:{}'.format(total, sum / total, sum))
        sum = 0

cuda:0
Net(
  (conv1): Conv2d(1, 3, kernel_size=(1, 3), stride=(1, 1))
  (conv2): Conv2d(3, 6, kernel_size=(2, 3), stride=(2, 1))
  (conv3): Conv2d(6, 10, kernel_size=(5, 3), stride=(1, 1))
  (fc1): Linear(in_features=550, out_features=4, bias=True)
)
total:174, accuracy:0.3045977011494253, sum:53
total_train:410, accuracy:0.25853658536585367, sum:106
Epoch[2], loss: 1.38788423
total:174, accuracy:0.3850574712643678, sum:67
total:174, accuracy:0.45977011494252873, sum:80
total_train:410, accuracy:0.35853658536585364, sum:147
Epoch[4], loss: 1.38563962
total:174, accuracy:0.47126436781609193, sum:82
total:174, accuracy:0.47701149425287354, sum:83
total_train:410, accuracy:0.3878048780487805, sum:159
Epoch[6], loss: 1.38348288
total:174, accuracy:0.47126436781609193, sum:82
total:174, accuracy:0.47126436781609193, sum:82
total_train:410, accuracy:0.3878048780487805, sum:159
Epoch[8], loss: 1.38134357
total:174, accuracy:0.47126436781609193, sum:82
total:174, accuracy:0.47701149425287354,

In [13]:
import random
index = random.sample(range(0,10),10)
print(index)

[0, 2, 4, 7, 3, 1, 9, 8, 6, 5]


In [29]:
torch.save(cnn.state_dict(), '\\net\\net_parameters.pkl')

In [45]:
for m in cnn.modules():
    params = list(m.named_parameters())
    print(params)

[('conv1.weight', Parameter containing:
tensor([[[[-0.3761,  0.1001, -0.4186, -0.1839]],

         [[-0.2533, -0.4018, -0.4310, -0.5048]],

         [[-0.2632, -0.3185,  0.1013, -0.0373]],

         [[-0.4070, -0.1267, -0.3912, -0.4335]],

         [[ 0.0283,  0.0544, -0.2606, -0.4131]],

         [[ 0.1456,  0.0818,  0.0535, -0.2685]],

         [[-0.2558, -0.1103,  0.2508,  0.1695]],

         [[ 0.0783,  0.1154,  0.0144, -0.0385]],

         [[ 0.4976,  0.2357,  0.4006,  0.4427]],

         [[ 0.2270, -0.0259, -0.1455, -0.0803]]],


        [[[-0.4097, -0.2975, -0.1239, -0.0020]],

         [[-0.1164, -0.2830,  0.0770, -0.1116]],

         [[ 0.1473, -0.4038, -0.2259, -0.3002]],

         [[-0.0664, -0.2939, -0.2580, -0.0312]],

         [[-0.2715,  0.0019, -0.0976, -0.1689]],

         [[-0.2187, -0.1226,  0.1654,  0.1052]],

         [[-0.3563,  0.1089,  0.1260, -0.1577]],

         [[ 0.2518, -0.2247,  0.2262, -0.1033]],

         [[ 0.1777,  0.5143,  0.2523,  0.4627]],

        